In [9]:
import nltk

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

from itertools import groupby
import pandas as pd

In [16]:
# Adapt those lines to your installation
jar_location = 'stanford-ner-2020-11-17/stanford-ner.jar'
model_location_3classes = 'stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
model_location_7classes = 'stanford-ner-2020-11-17/classifiers/english.muc.7class.distsim.crf.ser.gz'
st3 = StanfordNERTagger(model_location_3classes,jar_location,encoding='utf-8')
st7 = StanfordNERTagger(model_location_7classes,jar_location,encoding='utf-8')

In [15]:
text = '''Lovelace became close friends with her tutor Mary Somerville, who introduced her to Charles Babbage in 1833. She had a strong respect and affection for Somerville, and they corresponded for many years. Other acquaintances included the scientists Andrew Crosse, Sir David Brewster, Charles Wheatstone, Michael Faraday and the author Charles Dickens. She was presented at Court at the age of seventeen "and became a popular belle of the season" in part because of her "brilliant mind." By 1834 Ada was a regular at Court and started attending various events. She danced often and was able to charm many people, and was described by most people as being dainty, although John Hobhouse, Byron's friend, described her as "a large, coarse-skinned young woman but with something of my friend's features, particularly the mouth". This description followed their meeting on 24 February 1834 in which Ada made it clear to Hobhouse that she did not like him, probably because of the influence of her mother, which led her to dislike all of her father's friends. This first impression was not to last, and they later became friends.'''
tokenized_text = word_tokenize(text)
text_ner3 = st3.tag(tokenized_text)
text_ner7 = st7.tag(tokenized_text)

print("**** 7 classes ****")
for tag, chunk in groupby(text_ner7, lambda x:x[1]):
    if tag != "O":
        print("%-12s"%tag, " ".join(w for w, t in chunk))

**** 7 classes ****
PERSON       Mary Somerville
PERSON       Charles Babbage
DATE         1833
LOCATION     Somerville
PERSON       Andrew Crosse
PERSON       David Brewster
PERSON       Charles Wheatstone
PERSON       Michael Faraday
PERSON       Charles Dickens
DATE         1834
ORGANIZATION Ada
PERSON       John Hobhouse
PERSON       Byron
DATE         February 1834
ORGANIZATION Ada
PERSON       Hobhouse


In [42]:
import pandas as pd
import re

In [10]:
df = pd.read_csv('Cleaned_Ref_03.csv')
ss = ss = df['raw_ref'].values

In [45]:
s = 'This is a bad 2018'
match = re.search(r'[12]\d{3}', s)

if match:
    print(match.group(0))


2018


In [99]:
string = 'Vol 3, No.7'

def count_num(s):
    count = 0

    temp = list(s)
    for item in temp:
        if(item.isdigit()):
            count = count + 1
        else:
            pass
#    print (count)
    
    return count

In [91]:
#re.search(r'\d+', 'Vol 3, No.7').group()
re.sub('[^\d.,]' , '', 'Vol 3, No.7')

'3,.7'

In [ ]:
ref_list = []

for k in range (len(ss)):
#    match = re.search(r'[12]\d{3}', ss[k])
    if k < 324:
        continue
        
    ref_dict = {'date':[], 'names':[], 'title':[], 'raw_ref':[]}
        
    name_set = False
    date_set = False
    pos = -1    
        
    match = re.search(r'[12]\d{3}', ss[k])

    if match != None:
        ref_dict['date'] = int(match.group(0))
        date_set = True
            
        #print (ss[k])

        pos = ss[k].find(match.group(0))
                
        tokenized_text = word_tokenize(ss[k][:pos])
        text_ner3 = st3.tag(tokenized_text)
     #   text_ner7 = st7.tag(tokenized_text)
        
        for tag, chunk in groupby(text_ner3, lambda x:x[1]):
            if tag != "O":
                if tag == 'PERSON':
                    if len(ref_dict['names']) == 0: 
                        ref_dict['names'] =  " ".join(w for w, t in chunk)
                        name_set = True
                    else:
                        ref_dict['names'] = ref_dict['names'] + ', ' +  " ".join(w for w, t in chunk)

                    #print(k, "%-12s"%tag, " ".join(w for w, t in chunk))
                        
    if date_set == True:       
        if name_set == True:
            ref_dict['raw_ref'] = ss[k]
            ref_dict['names'] =  ref_dict['names'][:-1]

            ref_list.append(ref_dict)
        
        else: 
            print ('------- Try to Ignored', k, pos, name_set, date_set, ss[k])
            if pos != -1:
                s = ss[k][:pos].replace('(', '').strip()
                t = re.split(', |\.',s)
                print ('Split year', s)
                if len(t) > 2 and count_num(s) == 0:
                    print ('Found name', s)
                    ref_dict['raw_ref'] = ss[k]
                    ref_dict['names'] =  s
                
                else:
                    ssk = ss[k].split(':')

                    print ('Split :', ssk[0])
                    if len(ssk) > 1:
                        
                        t = re.split(', |\.',ssk[0])
                        if len(t) > 2:
                            print ('Found name', ssk[0])
                            ref_dict['raw_ref'] = ss[k]
                            ref_dict['names'] =  ssk[0]
    else:
         print ('------- Ignored', k, pos, name_set, date_set, ss[k])

------- Try to Ignored 324 75 False True [14] Holland, J.H.: Genetic algorithms. Scientific american 267(1), 66–73 (1992)
Split year [14] Holland, J.H.: Genetic algorithms. Scientific american 2671), 66–73
Split : [14] Holland, J.H.
Found name [14] Holland, J.H.
------- Try to Ignored 328 146 False True [18] Syswerda, G.: Uniform crossover in genetic algorithms. In: Proceedings of the Third International Conference on Genetic Algorithms, pp. 2–9 (1989). Morgan Kaufmann Publishers
Split year [18] Syswerda, G.: Uniform crossover in genetic algorithms. In: Proceedings of the Third International Conference on Genetic Algorithms, pp. 2–9
Split : [18] Syswerda, G.
Found name [18] Syswerda, G.
------- Try to Ignored 329 217 False True [19] Sehgal, A., Singandhupe, A., La, H.M., Tavakkoli, A., Louis, S.J.: Lidar-monocular visual odometry with genetic algorithm for parameter optimization. In: International Symposium on Visual Computing, pp. 358–370 (2019). Springer
Split year [19] Sehgal, A., S

------- Try to Ignored 494 32 False True [147] Proceedings of the EUROGEN2003 Conference: Evolutionary Methods for Design Optimization and Control with Applications to Industrial Problems, September 15– 17, 2003, Barcelona, Spain. ISBN: 8-4959-9933-1. Published on CD. See http:// congress.cimne.upc.es/eurogen03/ [accessed 2007-09-16].
Split year [147] Proceedings of the EUROGEN
Split : [147] Proceedings of the EUROGEN2003 Conference
------- Try to Ignored 517 49 False True [174], http:// icannga07.ee.pw.edu.pl/ [accessed 2007-08-31], and http://www.springerlink.com/ content/978-3-540-71590-0/ [accessed 2007-08-31]. [174] Bartlomiej Beliczyński, Andrzej Dzieliński, Marcin Iwanowski, and Bernardete Ribeiro, editors. Proceedings of Adaptive and Natural Computing Algorithms, 8th International Conference, ICANNGA 2007, Part II, volume 4432/2007 of Lecture Notes in Computer Science (LNCS), April 11–14, 2007, Warsaw University of Technology, Warsaw, Poland. Springer Berlin Heidelberg New Yo

------- Try to Ignored 695 181 False True [363] V. Černý. Thermodynamical approach to the traveling salesman problem: An eﬃcient simulation algorithm. Journal of Optimization Theory and Applications, 45(1):41–51, January 1985. ISSN: 0022-3239 (Print) 1573-2878 (Online). doi:10.1007/BF00940812. Communicated by S. E. Dreyfus. Online available at http://www.springerlink. com/content/gt0743622913gg33/fulltext.pdf [accessed 2008-03-26]. Also: Technical Report, Comenius University, Bratislava, Czechoslovakia, 1982.
Split year [363] V. Černý. Thermodynamical approach to the traveling salesman problem: An eﬃcient simulation algorithm. Journal of Optimization Theory and Applications, 451):41–51, January
Split : [363] V. Černý. Thermodynamical approach to the traveling salesman problem
Found name [363] V. Černý. Thermodynamical approach to the traveling salesman problem


In [76]:
s

['Doya, K.']

In [71]:
import re
t = 'Doya, K.'

re.split(', |\.',t)
#s = t.split(', |\.')
#re.split(',|.', t)

In [72]:
s

['Doya, K.']

In [51]:
len(ref_list)

1568

### ref_list